In [1]:
from fid import *

C:\Program Files\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [17]:
PARENT_DIR = 'fid'
labels = ['GAN', 'L2', 'L1*', 'CM', 'CM + L1*']
dir_labels = ['gan', 'l2', 'l1', 'cm', 'cml1']
inds_to_save = ['final'] + [20, 50, 100, 300]

inception_path = check_or_download_inception(None) # download inception network
create_inception_graph(inception_path)  # load the graph into the current TF graph

real_image_dir = os.path.join(PARENT_DIR, 'real')
mu_real, sigma_real = calculate_fid_stats(load_images_from_path(real_image_dir))

for painting_ind in inds_to_save:
    for dir_label in dir_labels:
        gen_image_dir = os.path.join(PARENT_DIR, str(painting_ind), dir_label)
        
        mu_gen, sigma_gen = calculate_fid_stats(load_images_from_path(gen_image_dir))
        
        try:
            fid = calculate_frechet_distance(mu_gen, sigma_gen, mu_real, sigma_real)
        
            print(painting_ind, dir_label, fid, sep="\t")
        except ValueError:
            print(painting_ind, dir_label, 'Value error', sep="\t")
    print()

final	gan	236.86584319334588
final	l2	237.83334160135124
final	l1	238.37351830902793
final	cm	239.47065507299615
final	cml1	240.00079906193736

20	gan	292.9496189964933
20	l2	270.8590301665218
20	l1	271.1785107945472
20	cm	284.44158083990374
20	cml1	287.1306029499479

50	gan	290.39986824796284
50	l2	287.3274114296746
50	l1	271.4467838896132
50	cm	293.2187315619593
50	cml1	273.0777704035577

100	gan	270.1127494788315
100	l2	270.8113169357256
100	l1	265.99857183342573
100	cm	277.20109861508064
100	cml1	262.7516269257586

300	gan	253.13286441742687
300	l2	256.3044992519167
300	l1	253.71944761654402
300	cm	259.8006705818094
300	cml1	251.19584564500303



In [1]:
calculate_fid('image', width=256, debug=True)

C:\Program Files\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


31 images found and loaded
create inception graph.. ok
calculte FID stats.. 

W0622 12:29:36.554729 47956 module_wrapper.py:139] From C:\Users\Peter\HumanoidPainter\fid.py:61: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0622 12:29:36.912511 47956 module_wrapper.py:139] From C:\Users\Peter\HumanoidPainter\fid.py:62: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.



finished
FID: -0.00012132218830629427


-0.00012132218830629427

In [1]:
# Help from https://github.com/bioinf-jku/TTUR/blob/master/fid.py
import os
import glob
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"#'0'
import numpy as np
import fid
#from scipy.misc import imread
import cv2
import tensorflow as tf

C:\Program Files\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Paths
image_path = 'image' # set path to some generated images
inception_path = fid.check_or_download_inception(None) # download inception network

# loads all images into memory (this might require a lot of RAM!)
image_list = glob.glob(os.path.join(image_path, '*.jpg'))

width = 256
images = np.zeros((len(image_list), width, width, 3))
i = 0
for img_fn in image_list:
    img = cv2.imread(img_fn, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (width, width))
    img = img.reshape(1, width, width, 3)
    # img = np.transpose(img, (0, 3, 1, 2))
    images[i] = img
    i += 1
    
print("%d images found and loaded" % len(images))
print(images.shape)

31 images found and loaded
(31, 256, 256, 3)


In [3]:
print("create inception graph..", end=" ", flush=True)
fid.create_inception_graph(inception_path)  # load the graph into the current TF graph
print("ok")

create inception graph.. ok


In [4]:
print("calculte FID stats..", end=" ", flush=True)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    mu_real, sigma_real = fid.calculate_activation_statistics(images, sess, batch_size=4)
    #np.savez_compressed(output_path, mu=mu, sigma=sigma)
print("finished")

calculte FID stats.. finished


In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    mu_gen, sigma_gen = fid.calculate_activation_statistics(images, sess, batch_size=4)

In [6]:
fid_value = fid.calculate_frechet_distance(mu_gen, sigma_gen, mu_real, sigma_real)
print("FID: %s" % fid_value)

FID: -0.00012132218830629427


In [7]:
def calculate_fid(images_path, width=256, debug=False):
    """
    Calculate the Frechet Inception Distance of images in a specified directory
    Based on: https://github.com/bioinf-jku/TTUR
    """
    inception_path = fid.check_or_download_inception(None) # download inception network

    # loads all images into memory (this might require a lot of RAM!)
    image_list = glob.glob(os.path.join(images_path, '*.jpg'))

    images = np.zeros((len(image_list), width, width, 3))
    i = 0
    for img_fn in image_list:
        img = cv2.imread(img_fn, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (width, width))
        img = img.reshape(1, width, width, 3)
        # img = np.transpose(img, (0, 3, 1, 2))
        images[i] = img
        i += 1
    if debug: print("%d images found and loaded" % len(images))
    
    if debug: print("create inception graph..", end=" ", flush=True)
    fid.create_inception_graph(inception_path)  # load the graph into the current TF graph
    if debug: print("ok")
    
    if debug: print("calculte FID stats..", end=" ", flush=True)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        mu_real, sigma_real = fid.calculate_activation_statistics(images, sess, batch_size=4)
    if debug: print("finished")
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        mu_gen, sigma_gen = fid.calculate_activation_statistics(images, sess, batch_size=4)
    
    fid_value = fid.calculate_frechet_distance(mu_gen, sigma_gen, mu_real, sigma_real)
    if debug: print("FID: %s" % fid_value)
    return fid_value

calculate_fid('image', width=256, debug=True)

31 images found and loaded
create inception graph.. ok
calculte FID stats.. finished
FID: -0.00012196153500099172


-0.00012196153500099172